In [1]:
import regex as re
import nltk
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_squared_error as MSE, r2_score
from scipy.stats import skew,kurtosis,zscore
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [ ]:
!unzip "/content/Restraunt_MH.zip"

Archive:  /content/Restraunt_MH.zip
replace Participants_Data_Final/Data_Train.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Participants_Data_Final/Data_Train.xlsx  
replace __MACOSX/Participants_Data_Final/._Data_Train.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/Participants_Data_Final/._Data_Train.xlsx  
replace Participants_Data_Final/Data_Test.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Participants_Data_Final/Data_Test.xlsx  
replace __MACOSX/Participants_Data_Final/._Data_Test.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/Participants_Data_Final/._Data_Test.xlsx  
replace Participants_Data_Final/Sample_submission.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Participants_Data_Final/Sample_submission.xlsx  
replace __MACOSX/Participants_Data_Final/._Sample_submission.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/Participants_Data_Final/._Sample_submission.xlsx  


# Prediction

In [2]:
df = pd.read_excel("Data_Train.xlsx")

In [3]:
df.head(2)

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes,1200
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes,1500


In [4]:
df['COST'].skew()

3.9788007578247573

In [5]:
df = df.drop(['RESTAURANT_ID'],1)

In [6]:
y = df.COST

In [7]:
df.isnull().values.any()

True

In [8]:
df.isnull().sum()

TITLE          0
CUISINES       0
TIME           0
CITY         112
LOCALITY      98
RATING         2
VOTES       1204
COST           0
dtype: int64

In [9]:
df['CITY'].fillna('missingcity', inplace=True)
df['LOCALITY'].fillna('missinglocality', inplace=True)
df['VOTES'].fillna('0 votes', inplace=True)
df = df.fillna(0)

In [10]:
df.isnull().sum()

TITLE       0
CUISINES    0
TIME        0
CITY        0
LOCALITY    0
RATING      0
VOTES       0
COST        0
dtype: int64

In [11]:
df['VOTES'] = df['VOTES'].apply(lambda x: x.split('votes')[0])

In [12]:
df['RATING'] = df['RATING'].replace('-', '0')
df['RATING'] = df['RATING'].replace('NEW', '0')

In [13]:
df_num = df[['VOTES','RATING']]
df_num.head(2)

,VOTES,RATING
0,49,3.6
1,30,4.2


In [14]:
print(*df_num['RATING'])

3.6 4.2 3.8 4.1 3.8 4.0 4.1 4.3 3.9 3.3 4.1 4.0 3.4 0 4.0 4.5 3.6 3.5 3.6 4.1 4.2 4.0 4.0 3.9 4.2 3.8 3.4 4.1 4.3 4.2 3.5 4.0 4.0 4.4 2.7 3.3 3.7 4.3 4.0 3.4 3.3 3.8 3.3 4.0 4.2 4.1 4.0 3.9 4.7 4.3 3.3 4.0 3.9 3.7 3.7 3.8 3.3 3.6 4.0 3.9 3.6 3.6 0 4.0 3.7 4.3 3.4 4.3 3.3 0 4.0 3.6 4.3 4.0 4.3 3.5 3.1 3.4 3.9 3.8 4.0 3.8 4.0 4.2 4.0 0 3.8 3.1 3.7 4.1 4.1 4.0 3.9 3.4 3.9 4.2 3.8 3.9 3.8 3.7 3.8 3.4 3.5 4.0 2.7 4.3 4.2 3.4 4.0 4.1 3.7 0 3.7 3.6 4.1 4.0 3.4 0 4.4 3.8 0 3.6 3.6 4.2 3.3 4.3 3.1 3.9 3.7 3.6 4.2 3.9 4.0 3.8 3.9 3.5 3.9 4.1 3.8 3.8 3.5 3.8 3.7 4.0 3.9 4.7 3.7 4.2 4.0 3.5 4.2 3.3 3.8 4.0 3.7 4.0 0 3.7 4.2 2.5 3.8 3.7 3.4 3.9 3.5 0 3.7 4.1 4.1 3.1 4.3 3.9 4.0 3.4 4.2 3.8 4.1 4.3 0 4.5 3.5 3.1 3.5 4.0 4.1 3.1 4.6 4.1 3.9 2.8 3.9 3.5 3.9 4.3 0 4.3 4.6 3.8 3.7 4.0 3.6 3.8 4.1 3.7 0 3.6 4.7 4.7 3.9 4.1 3.9 3.8 0 4.1 0 0 3.7 3.8 3.8 4.0 3.6 3.5 3.5 3.9 3.7 3.8 3.7 0 3.6 3.3 4.0 4.2 3.0 3.6 3.8 0 4.0 4.0 4.0 2.8 3.6 3.6 2.7 4.0 3.9 4.1 3.4 3.8 3.3 3.7 3.4 3.6 3.8 0 3.7 4.3 3.2 3.8 3.9 

In [15]:
df_txt = df.drop(['VOTES','RATING'],1)
df_txt.head(2)

,TITLE,CUISINES,TIME,CITY,LOCALITY,COST
0,CASUAL DINING,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,1200
1,"CASUAL DINING,BAR","Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,1500


In [16]:
df_txt['main'] = df_txt[df_txt.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

df_txt.main.head(3)

0    CASUAL DINING,Malwani, Goan, North Indian,11am...
1    CASUAL DINING,BAR,Asian, Modern Indian, Japane...
2    CASUAL DINING,North Indian, Chinese, Biryani, ...
Name: main, dtype: object

In [17]:
X_text = df_txt.main

In [18]:
import regex as re
model = []
for i in range(0, df.shape[0]):
    data = re.sub(r'\W', ' ', str(X_text[i]))
    data = data.lower()
    data = re.sub(r'\s+[a-z]\s+', ' ',data)
    data = re.sub(r'\s+', ' ', data)
    model.append(data)   

In [19]:
model[0]

'casual dining malwani goan north indian 11am 4pm 7 30pm 11 30pm mon sun thane dombivali east 1200'

In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
vector = TfidfVectorizer(max_features = 2000, min_df = 1, max_df = 0.99, stop_words = stopwords.words('english'))
X_text = vector.fit_transform(model).toarray()

In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [23]:
df_num = scaler.fit_transform(df_num,1)
df_num

array([[1.18972466e-03, 7.34693878e-01],
       [7.28402855e-04, 8.57142857e-01],
       [5.36590103e-03, 7.75510204e-01],
       ...,
       [1.47622979e-02, 8.16326531e-01],
       [7.76963046e-04, 7.14285714e-01],
       [1.64376244e-02, 6.93877551e-01]])

In [24]:
X_num = df_num

In [25]:
X = np.concatenate((X_text,X_num),axis = 1)
X

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.18972466e-03, 7.34693878e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 7.28402855e-04, 8.57142857e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 5.36590103e-03, 7.75510204e-01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.47622979e-02, 8.16326531e-01],
       [0.00000000e+00, 0.00000000e+00, 2.26034763e-01, ...,
        0.00000000e+00, 7.76963046e-04, 7.14285714e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.64376244e-02, 6.93877551e-01]])

# Fitting

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = 42)

In [27]:
from sklearn.ensemble import RandomForestRegressor

In [28]:
regr = RandomForestRegressor(n_jobs = -1, random_state = 42, n_estimators = 100, verbose = 2)

In [29]:
regr.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   37.0s


building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.6min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=2, warm_start=False)

In [30]:
import numpy as np

pred1 = regr.predict(X_test)
pred2 = regr.predict(X_train) 
rmse = np.sqrt(MSE(y_test, pred1)) 
print("RMSE : % f" %(rmse)) 

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


RMSE :  98.302448


[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.2s finished


In [31]:
print(r2_score(y_test,pred1)*100)
print(r2_score(y_train,pred2)*100)

97.35512309319328
98.46302471805905


In [32]:
import lightgbm as lgb
train = lgb.Dataset(X_train, label=y_train)
test = lgb.Dataset(X_test, label=y_test)

In [33]:
param = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l2', 'auc'],
   'num_iterations': 100,
         'learning_rate': 0.01,  
         'num_leaves': 65,
         'max_depth': 60, 
         'min_data_in_leaf': 1, 
         'max_bin': 35, 
         'min_data_in_bin':10, 
         'random_state' : 42 
}

In [34]:
gbm = lgb.LGBMRegressor(**param)

In [35]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds = 100)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l1: 401.482	valid_0's auc: 1	valid_0's l2: 359162
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's l1: 398.042	valid_0's auc: 1	valid_0's l2: 352864
[3]	valid_0's l1: 394.61	valid_0's auc: 1	valid_0's l2: 346669
[4]	valid_0's l1: 391.27	valid_0's auc: 1	valid_0's l2: 340664
[5]	valid_0's l1: 388.047	valid_0's auc: 1	valid_0's l2: 334813
[6]	valid_0's l1: 384.762	valid_0's auc: 1	valid_0's l2: 329036
[7]	valid_0's l1: 381.56	valid_0's auc: 1	valid_0's l2: 323408
[8]	valid_0's l1: 378.391	valid_0's auc: 1	valid_0's l2: 317928
[9]	valid_0's l1: 375.27	valid_0's auc: 1	valid_0's l2: 312574
[10]	valid_0's l1: 372.184	valid_0's auc: 1	valid_0's l2: 307320
[11]	valid_0's l1: 369.091	valid_0's auc: 1	valid_0's l2: 302156
[12]	valid_0's l1: 365.958	valid_0's auc: 1	valid_0's l2: 296978
[13]	valid_0's l1: 362.955	valid_0's auc: 1	valid_0's l2: 292033
[14]	valid_0's l1: 360	valid_0's auc: 1	valid_0's l2: 287159
[15]	valid_0's l1: 357.042	valid_0's auc: 1	

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.01, max_bin=35,
              max_depth=60, metric=['l2', 'auc'], min_child_samples=20,
              min_child_weight=0.001, min_data_in_bin=10, min_data_in_leaf=1,
              min_split_gain=0.0, n_estimators=100, n_jobs=-1,
              num_iterations=100, num_leaves=65, objective='regression',
              random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
              task='train')

In [36]:
pred01 = gbm.predict(X_test, num_iteration=gbm.best_iteration_) 
rmse = np.sqrt(MSE(y_test, pred1)) 
print("RMSE : % f" %(rmse)) 

RMSE :  98.302448


In [37]:
r2_score(y_test,pred01)*100

78.25389605725925

# Test Set

In [39]:
df = pd.read_excel("Data_Test.xlsx")

In [40]:
df.head(2)

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES
0,CASUAL DINING,4085,"North Indian, Chinese, Mughlai, Kebab",12noon – 12midnight (Mon-Sun),Noida,Sector 18,4.3,564 votes
1,QUICK BITES,12680,"South Indian, Fast Food, Pizza, North Indian",7am – 12:30AM (Mon-Sun),Mumbai,Grant Road,4.2,61 votes


In [41]:
df = df.drop(['RESTAURANT_ID'],1)

In [42]:
df.isnull().values.any()

True

In [43]:
df.isnull().sum()

TITLE         0
CUISINES      0
TIME          0
CITY         35
LOCALITY     30
RATING        2
VOTES       402
dtype: int64

In [44]:
df['CITY'].fillna('missingcity', inplace=True)
df['LOCALITY'].fillna('missinglocality', inplace=True)
df['VOTES'].fillna('0 votes', inplace=True)
df = df.fillna(0)

In [45]:
df.isnull().sum()

TITLE       0
CUISINES    0
TIME        0
CITY        0
LOCALITY    0
RATING      0
VOTES       0
dtype: int64

In [46]:
df['VOTES'] = df['VOTES'].apply(lambda x: x.split('votes')[0])

In [47]:
df['RATING'] = df['RATING'].replace('-', '0')
df['RATING'] = df['RATING'].replace('NEW', '0')

In [48]:
df_num = df[['VOTES','RATING']]
df_num.head(2)

,VOTES,RATING
0,564,4.3
1,61,4.2


In [49]:
df_txt = df.drop(['VOTES','RATING'],1)
df_txt.head(2)

,TITLE,CUISINES,TIME,CITY,LOCALITY
0,CASUAL DINING,"North Indian, Chinese, Mughlai, Kebab",12noon – 12midnight (Mon-Sun),Noida,Sector 18
1,QUICK BITES,"South Indian, Fast Food, Pizza, North Indian",7am – 12:30AM (Mon-Sun),Mumbai,Grant Road


In [50]:
df_txt['main'] = df_txt[df_txt.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

df_txt.main.head(3)

0    CASUAL DINING,North Indian, Chinese, Mughlai, ...
1    QUICK BITES,South Indian, Fast Food, Pizza, No...
2    CASUAL DINING,North Indian, Seafood, Biryani, ...
Name: main, dtype: object

In [51]:
X_text = df_txt.main

In [52]:
import regex as re
model = []
for i in range(0, df.shape[0]):
    data = re.sub(r'\W', ' ', str(X_text[i]))
    data = data.lower()
    data = re.sub(r'\s+[a-z]\s+', ' ',data)
    data = re.sub(r'\s+', ' ', data)
    model.append(data)   

In [53]:
model[0]

'casual dining north indian chinese mughlai kebab 12noon 12midnight mon sun noida sector 18'

In [54]:
#vector = TfidfVectorizer(max_features = 4000, min_df = 1, max_df = 0.8, stop_words = stopwords.words('english'))
X_text = vector.transform(model).toarray()

In [55]:
X_num = scaler.transform(df_num)
X_num

array([[0.01369397, 0.87755102],
       [0.00148109, 0.85714286],
       [0.00849803, 0.7755102 ],
       ...,
       [0.00128685, 0.75510204],
       [0.        , 0.        ],
       [0.00152965, 0.75510204]])

In [56]:
X = np.concatenate((X_text,X_num),axis = 1)
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.01369397,
        0.87755102],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00148109,
        0.85714286],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00849803,
        0.7755102 ],
       ...,
       [0.        , 0.        , 0.22856814, ..., 0.        , 0.00128685,
        0.75510204],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00152965,
        0.75510204]])

In [57]:
predictions = regr.predict(X)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished


In [58]:
predictions

array([633.  , 357.71, 530.  , ..., 353.96, 338.06, 366.5 ])

In [59]:
df_sub = pd.DataFrame(data = predictions, columns=['COST'])

In [60]:
df_sub.to_csv('result_res.csv',index =False)